In [1]:
import zipfile
from pulp import LpProblem
import os
import gzip
from gurobipy import read, GRB

import graph_nets as gn
import sonnet as snt
import tensorflow as tf
from graph_nets import graphs
from graph_nets import utils_tf

import numpy as np

from pyscipopt import Model, quicksum, SCIP_RESULT, SCIP_PARAMSETTING,SCIP_EVENTTYPE,Eventhdlr

#temp_path = 'data/extract/bm_ex/30n20b8.mps'

In [2]:
temp_path = 'data/extract/bm_ex/50v-10.mps'

model = Model()
model.readProblem(temp_path)
model.optimize()


original problem has 2013 variables (1464 bin, 183 int, 0 impl, 366 cont) and 233 constraints
presolving:
(round 1, fast)       0 del vars, 0 del conss, 0 add conss, 366 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
   (0.0s) running MILP presolver
   (0.0s) MILP presolver found nothing
   (0.0s) probing: 51/1464 (3.5%) - 0 fixings, 0 aggregations, 0 implications, 0 bound changes
   (0.0s) probing aborted: 50/50 successive totally useless probings
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetry present (symcode time: 0.00)
presolving (2 rounds: 2 fast, 1 medium, 1 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 366 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 2013 variables (1464 bin, 183 int, 0 impl, 366 cont) and 233 constraints
    233 constraints of type <linear>
Presolving Time: 0

In [2]:
def collect_every_solution(path):
    model = Model()
    model.readProblem(path)

    # Disable presolving and enable solution collection
    model.setPresolve(SCIP_PARAMSETTING.OFF)
    #model.setParam('limits/solutions', -1)  # Collect all solutions



    class SolutionCollector(Eventhdlr):
        def __init__(self):
            super().__init__()
            self.model = model
            self.solutions = []

        def eventinit(self):
            self.model.catchEvent(SCIP_EVENTTYPE.BESTSOLFOUND, self)

        def eventexit(self):
            self.model.dropEvent(SCIP_EVENTTYPE.BESTSOLFOUND, self)

        def eventexec(self, event):
            #sol = self.model.getBestSol()
            #x_val = self.model.getVars()

           # self.solutions.append(x_val)
            
            sol = self.model.getBestSol()
            sol_values = {var.name: self.model.getSolVal(sol, var) for var in self.model.getVars()}
            self.solutions.append(sol_values)


    # Create an instance of the event handler
    solution_collector = SolutionCollector()

    # Include the event handler in the model
    model.includeEventhdlr(solution_collector, "SolutionCollector", "Collects all feasible solutions")

    # Solve the model
    model.optimize()


    sol_set = solution_collector.solutions

    #现在我们拥有了路径中问题的所有可行解的集合，可以用它来构建二部图了


    return sol_set

In [3]:
ans = collect_every_solution(temp_path)





original problem has 18380 variables (18318 bin, 62 int, 0 impl, 0 cont) and 576 constraints
presolving:
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetry present (symcode time: 0.00)
presolving (0 rounds: 0 fast, 0 medium, 0 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 18380 variables (18318 bin, 62 int, 0 impl, 0 cont) and 576 constraints
    576 constraints of type <linear>
transformed objective value is always integral (scale: 1)
Presolving Time: 0.04

 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   | primalbound  |  gap   | compl. 
  1.0s|     1 |     0 |  6371 |     - |   117M |   0 |  18k| 527 | 527 |   0 |  0 |   0 |   0 | 4.333557e+01 |      --      |    Inf | unknown
  1.2s|     1 |     0 | 

In [8]:
print(ans[0])
#looks like evevry problem get a same output index



{'x2024': 0.0, 'x226': 0.0, 'x227': 0.0, 'x228': 0.0, 'x229': 0.0, 'x230': 0.0, 'x231': 0.0, 'x232': 0.0, 'x233': 0.0, 'x234': 0.0, 'x235': 0.0, 'x236': 0.0, 'x237': 4.440892098500628e-16, 'x238': 0.0, 'x239': 0.0, 'x240': 0.0, 'x241': 0.0, 'x242': 0.0, 'x243': 0.0, 'x244': 0.0, 'x245': 0.0, 'x246': 0.0, 'x247': 0.0, 'x248': 0.0, 'x249': 0.0, 'x250': 0.0, 'x251': 0.0, 'x252': 0.0, 'x253': 0.0, 'x254': 0.0, 'x255': 0.0, 'x256': 0.9999999999999996, 'x257': -3.944304526105059e-31, 'x258': 0.0, 'x259': 1.9721522630525295e-31, 'x260': 0.0, 'x261': 0.0, 'x262': 0.0, 'x263': 0.0, 'x264': 0.0, 'x265': 0.0, 'x266': 0.0, 'x267': 0.0, 'x268': 0.0, 'x269': 0.0, 'x270': 0.0, 'x271': 0.0, 'x272': 0.0, 'x273': 0.0, 'x274': 0.0, 'x275': 0.0, 'x276': 0.0, 'x277': 0.0, 'x278': 0.0, 'x279': 0.0, 'x280': 0.0, 'x281': 0.0, 'x282': 0.0, 'x283': 0.0, 'x284': 0.0, 'x285': 0.0, 'x286': 0.0, 'x287': 0.0, 'x288': 0.0, 'x289': 0.0, 'x290': 0.0, 'x291': 0.0, 'x292': 0.0, 'x293': 0.0, 'x294': 0.0, 'x295': 0.0, 'x29

In [9]:
print(ans[1])

{'x2024': 0.0, 'x226': 0.0, 'x227': 0.0, 'x228': 0.0, 'x229': 0.0, 'x230': 0.0, 'x231': 0.0, 'x232': 0.0, 'x233': 0.0, 'x234': 0.0, 'x235': 0.0, 'x236': 0.0, 'x237': 0.0, 'x238': 0.0, 'x239': 0.0, 'x240': 0.0, 'x241': 0.0, 'x242': 0.0, 'x243': -1.9984014443252818e-15, 'x244': 0.0, 'x245': 0.0, 'x246': 0.0, 'x247': 0.0, 'x248': 0.0, 'x249': 0.0, 'x250': 0.0, 'x251': 0.0, 'x252': 0.0, 'x253': 0.0, 'x254': 0.0, 'x255': 0.0, 'x256': 0.0, 'x257': 0.0, 'x258': 0.0, 'x259': 1.000000000000002, 'x260': 0.0, 'x261': 0.0, 'x262': 0.0, 'x263': 0.0, 'x264': 0.0, 'x265': 0.0, 'x266': 0.0, 'x267': 0.0, 'x268': 0.0, 'x269': 0.0, 'x270': 0.0, 'x271': 0.0, 'x272': 0.0, 'x273': 0.0, 'x274': 0.0, 'x275': 0.0, 'x276': 0.0, 'x277': 0.0, 'x278': 0.0, 'x279': 0.0, 'x280': 0.0, 'x281': 0.0, 'x282': 0.0, 'x283': 0.0, 'x284': 0.0, 'x285': 0.0, 'x286': 0.0, 'x287': 0.0, 'x288': 0.0, 'x289': 0.0, 'x290': 0.0, 'x291': 0.0, 'x292': 0.0, 'x293': 0.0, 'x294': 0.0, 'x295': 0.0, 'x296': 0.0, 'x297': 0.0, 'x298': 0.0, 'x

In [56]:
#first we get the nodes of the model
def get_nodes(path):
    model = Model()
    model.readProblem(path)

    # Disable presolving and enable solution collection
    model.setPresolve(SCIP_PARAMSETTING.OFF)
    #model.setParam('limits/solutions', -1)  # Collect all solutions
    
    
    nodes=[]
    #put variables to nodes
    nodes_count=0
    variables = model.getVars()
    for var in variables:
        coeff = var.getObj() # 获取变量在目标函数中的系数
        #print(f"变量 {var.name} 在目标函数中的系数为 {coeff}")
        nodes.append([coeff,0])
        nodes_count+=1
        
    node_features = np.array([[feat1, feat2] for feat1, feat2 in nodes])
    
    #put constraints to nodes
    cons_list=[]
    constraints = model.getConss()
    for cons in constraints:
        rhs = model.getRhs(cons)
        #print(f"约束 {cons.name} 的 RHS 为: {rhs}")
        cons_list.append([0,rhs])
    constraint_features= np.array([[con1, con2] for con1, con2 in cons_list])
        
    #print(node_features)
    
    combined_features = np.vstack((node_features, constraint_features))
    return combined_features







In [57]:
def get_index_to_nodes(path):
    """
    我们需要将每个变量和index对应起来
    """
    model = Model()
    model.readProblem(path)

    # Disable presolving and enable solution collection
    model.setPresolve(SCIP_PARAMSETTING.OFF)
    #model.setParam('limits/solutions', -1)  # Collect all solutions
    
    names=[]
    #put variables to nodes
    nodes_count=0
    variables = model.getVars()
    for var in variables:
        
        names.append(var.name)
    
    
    constraints = model.getConss()
    for cons in constraints:
        
        names.append(cons.name)
    return names

name_list = get_index_to_nodes(temp_path)

#print(name_list[2025])
        
    

original problem has 2025 variables (1800 bin, 0 int, 0 impl, 225 cont) and 230 constraints


In [58]:
def get_edges(path):
    model = Model()
    model.readProblem(path)

    # Disable presolving and enable solution collection
    model.setPresolve(SCIP_PARAMSETTING.OFF)
    #model.setParam('limits/solutions', -1)  # Collect all solutions
    
    #获取nodes列表index和节点的关系
    name_list = get_index_to_nodes(path)
    
    #初始化 senders, receivers, 和 edges 列表
    senders = []
    receivers = []
    edges = []

    
    constraints = model.getConss()
    
    for cons in constraints:
        
        #获得了这个cons在name_list里的index
        cons_index = name_list.index(cons.name)
        
        coef_of_cons = model.getValsLinear(cons)
        
        data_dict = {name_list.index(key): value for key, value in coef_of_cons.items()}
        
        for receiver, weight in data_dict.items():
            senders.append(cons_index)  # 发送者节点 ID
            receivers.append(receiver)  # 接收者节点 ID
            edges.append(weight)  # 边的权重
        
    return senders, receivers, edges
            
    #print("Coefficients of variables in the constraint:")
    #print(coefficients)
    
senders,receivers,edges = get_edges(temp_path)

    

original problem has 2025 variables (1800 bin, 0 int, 0 impl, 225 cont) and 230 constraints
original problem has 2025 variables (1800 bin, 0 int, 0 impl, 225 cont) and 230 constraints


In [59]:
nodes = get_nodes(temp_path)
n_node = len(nodes)
global_features = np.zeros((1, 1))
senders,receivers,edges = get_edges(temp_path)
n_edge = len(edges)

graph_tuple_test = graphs.GraphsTuple(
    nodes=nodes,
    edges=np.array(edges).reshape(-1, 1),
    globals=global_features,
    senders=np.array(senders),
    receivers=np.array(receivers),
    n_node=np.array([len(nodes)]),
    n_edge=np.array([len(edges)])
)



original problem has 2025 variables (1800 bin, 0 int, 0 impl, 225 cont) and 230 constraints
original problem has 2025 variables (1800 bin, 0 int, 0 impl, 225 cont) and 230 constraints
original problem has 2025 variables (1800 bin, 0 int, 0 impl, 225 cont) and 230 constraints


Nodes: [[23.  0.]
 [ 3.  0.]
 [ 5.  0.]
 ...
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]]
Edges: [[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]
Globals: [[0.]]
Senders: [2025 2025 2025 ... 2254 2254 2254]
Receivers: [1800 1802 1804 ... 1798    0 1799]
Number of nodes in each graph: [2255]
Number of edges in each graph: [12150]
